In [45]:
from ultralytics import YOLO
import cv2
import numpy as np
import requests
import re
from datetime import datetime
import json
import os
from twilio.rest import Client

In [46]:
import torch

In [47]:
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)
if cuda_available:
    print("Number of GPUs available:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print("GPU", i, ":", torch.cuda.get_device_name(i))
else:
    print("No GPUs available, CPU will be used.")

CUDA available: True
Number of GPUs available: 1
GPU 0 : NVIDIA GeForce GTX 1650


In [48]:
with open('config_parking.json', 'r') as f:
    config = json.load(f)
input_path = config['rtsp_urls']


In [49]:
# Load the credentials from the JSON file
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

API_TOKEN = credentials['API_TOKEN']
PR_API_URL = credentials['PR_API_URL']

# Twilio credentials and setup
account_sid = credentials['account_sid']
auth_token = credentials['auth_token']

In [50]:
client = Client(account_sid, auth_token)

In [51]:
model = YOLO('yolov5mu.pt')
class_names = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
    "toothbrush"
]

In [52]:
if os.path.isfile(input_path):
    is_image = input_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))
    if is_image:
        frame = cv2.imread(input_path)
        if frame is None:
            print("Error: Unable to load image")
            exit()
    else:
        cap = cv2.VideoCapture(input_path)
else:
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Unable to open video stream")
        exit()


In [53]:
parking_log = {}
processed_cars = set() 

In [54]:
def format_license_plate(plate_number):
    plate_number = plate_number.upper()
    match = re.match(r"([A-Z]{2})(\d{2})([A-Z]{2})(\d{4})", plate_number)
    
    if match:
        formatted_plate = f"{match.group(1)} {match.group(2)} {match.group(3)} {match.group(4)}"
        return formatted_plate
    else:
        return plate_number 

In [55]:
# Twilio notification function
def send_notification(plate, park_time, entry=True):
    message_body = f"Vehicle with Registration No.: {plate} has {'entered' if entry else 'exited'} SP CAR PARKING At {park_time}. Thank You"
    message = client.messages.create(
        body=message_body,
        from_='whatsapp:+14155238886',  # Twilio number
        to='whatsapp:+919842138883'  # Your phone number to receive notifications
    )
    print(f"Notification sent: {message_body}")

In [56]:
# Function to log car entry and send notification
def log_car_entry(plate):
    entry_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    parking_log[plate] = {'entry': entry_time}
    print(f"Car {plate} entered at {entry_time}")
    send_notification(plate, entry_time, entry=True)  # Send entry notification

# Function to log car exit and send notification
def log_car_exit(plate):
    exit_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    if plate in parking_log:
        parking_log[plate]['exit'] = exit_time
        print(f"Car {plate} exited at {exit_time}")
        send_notification(plate, exit_time, entry=False)  # Send exit notification

In [57]:
# Function to recognize license plate using Plate Recognizer API
def recognize_plate(car_image):
    success, image_jpg = cv2.imencode('.jpg', car_image)
    files = {'upload': image_jpg.tobytes()}
    headers = {'Authorization': f'Token {API_TOKEN}'}

    # Send the image to Plate Recognizer
    response = requests.post(PR_API_URL, headers=headers, files=files)

    if response.status_code == 201:  # 201 indicates successful processing
        data = response.json()
        if len(data['results']) > 0:
            # Extract the main license plate
            plate_number = data['results'][0]['plate']
            
            # Format the plate number
            formatted_plate = format_license_plate(plate_number)
            
            
            
            # Print the detected plate and timestamp
            print(f"License Plate Detected: {formatted_plate}")
            
            
            # Check if it's a new car (entry) or existing car (exit)
            if formatted_plate not in parking_log:
                log_car_entry(formatted_plate)
            else:
                log_car_exit(formatted_plate)
            
            return formatted_plate
        else:
            print("No plate detected.")
    else:
        print(f"Error: {response.status_code}, {response.text}")
    return None, None


In [58]:
THRESHOLD_LINE_Y = 580  
# Function to check if the car has crossed the threshold line
def has_crossed_threshold(y2):
    return y2 > THRESHOLD_LINE_Y

In [59]:
def preprocess_image(image):
    return image  

In [60]:
# Resize the video frame while maintaining aspect ratio
def resize_frame(frame, width, height):
    h, w, _ = frame.shape
    aspect_ratio = w / h
    if w > h:
        new_width = width
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = height
        new_width = int(new_height * aspect_ratio)
    return cv2.resize(frame, (new_width, new_height))

In [61]:
def process_image(image):
    results = model(image)
    for result in results:
        boxes = result.boxes.xyxy  
        classes = result.boxes.cls  

        for i, box in enumerate(boxes):
            class_id = int(classes[i])
            if class_names[class_id] == "car":  
                x1, y1, x2, y2 = map(int, box)  
                car_region = image[y1:y2, x1:x2]  
                car_region = preprocess_image(car_region)
                recognize_plate(car_region)
    
    annotated_frame = results[0].plot()  # Visualize the results
    cv2.imshow('YOLOv8 Inference', annotated_frame)
    cv2.waitKey(0)  # Wait for key press to close the image window

In [62]:
# def capture_and_detect():
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("Failed to retrieve frame from camera or video")
#             break
        
#         # Resize the frame to fit the full-screen window
#         frame_resized = resize_frame(frame, 1080, 720)
        
#         # Run YOLO detection
#         results = model(frame_resized)

#         if results and len(results) > 0:
#             # If YOLO detects objects, annotate the frame
#             annotated_frame = results[0].plot()
#         else:
#             # If no detections, use the resized frame as the annotated frame
#             annotated_frame = frame_resized.copy()
        
#         # Check for car detections and process them
#         for result in results:
#             boxes = result.boxes.xyxy  # Get the bounding boxes
#             classes = result.boxes.cls  # Get the class labels

#             for i, box in enumerate(boxes):
#                 class_id = int(classes[i])
#                 if class_names[class_id] == "car":  # Only process cars
#                     x1, y1, x2, y2 = map(int, box)  # Extract bounding box
                    
#                     # Check if the car has crossed the Y-threshold line
#                     if has_crossed_threshold(y2):
#                         car_id = (x1, y1, x2, y2)

#                         # If this car has not been processed yet, process it
#                         if car_id not in processed_cars:
#                             car_region = frame_resized[y1:y2, x1:x2]  # Crop the car region
#                             # recognize_plate(car_region)  # Send the cropped image for recognition
#                             processed_cars.add(car_id)
                        
#         # Draw the threshold line on the annotated frame
#         cv2.line(annotated_frame, (0, THRESHOLD_LINE_Y), (annotated_frame.shape[1], THRESHOLD_LINE_Y), (0, 255, 0), 2)  # Green line

#         # Visualize the result
#         cv2.imshow('YOLOv8 Inference', annotated_frame)

#         # Press 'q' to exit the loop
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# # Initialize video capture
# cap = cv2.VideoCapture(input_path)
# if not cap.isOpened():
#     print("Error: Unable to open video stream")
#     exit()

# # Set the window to full screen or specific size
# #cv2.namedWindow('YOLOv8 Inference', cv2.WINDOW_NORMAL)
# #cv2.setWindowProperty('YOLOv8 Inference', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)  # For full-screen display



In [63]:
# Function to compute image sharpness using Laplacian variance
def calculate_sharpness(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()  # Calculate the variance of the Laplacian
    return laplacian_var

In [64]:
# def capture_and_detect():
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("Failed to retrieve frame from camera or video")
#             break
        
#         # Resize the frame to fit the full-screen window
#         frame_resized = resize_frame(frame, 1080, 720)
        
#         # Run YOLO detection
#         results = model(frame_resized)

#         if results and len(results) > 0:
#             # If YOLO detects objects, annotate the frame
#             annotated_frame = results[0].plot()
#         else:
#             # If no detections, use the resized frame as the annotated frame
#             annotated_frame = frame_resized.copy()
        
#         # Check for car detections and process them
#         for result in results:
#             boxes = result.boxes.xyxy  # Get the bounding boxes
#             classes = result.boxes.cls  # Get the class labels

#             for i, box in enumerate(boxes):
#                 class_id = int(classes[i])
#                 if class_names[class_id] == "car":  # Only process cars
#                     x1, y1, x2, y2 = map(int, box)  # Extract bounding box
                    
#                     # Check if the car has crossed the Y-threshold line
#                     if has_crossed_threshold(y2):
#                         car_id = (x1, y1, x2, y2)

#                         # If this car has not been processed yet, process it
#                         if car_id not in processed_cars:
#                             car_region = frame_resized[y1:y2, x1:x2]  # Crop the car region

#                             # Display the cropped car image in a new window for testing
#                             cv2.imshow("Cropped Car", car_region)
#                             print("Car has crossed the threshold. Showing cropped car image.")
                            
#                             # Wait for a key press to close the cropped car window
#                             cv2.waitKey(0)
#                             cv2.destroyWindow("Cropped Car")  # Close the cropped car window after key press

#                             # Add the car to the processed set
#                             processed_cars.add(car_id)
                        
#         # Draw the threshold line on the annotated frame
#         cv2.line(annotated_frame, (0, THRESHOLD_LINE_Y), (annotated_frame.shape[1], THRESHOLD_LINE_Y), (0, 255, 0), 2)  # Green line

#         # Visualize the result
#         cv2.imshow('YOLOv8 Inference', annotated_frame)

#         # Press 'q' to exit the loop
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()


In [65]:

def capture_and_detect():
    car_detected = False  # Flag to indicate when the car has touched the threshold

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to retrieve frame from camera or video")
            break
        
        # Resize the frame to fit the full-screen window
        frame_resized = resize_frame(frame, 1080, 720)
        
        # Run YOLO detection
        results = model(frame_resized)

        if results and len(results) > 0:
            # If YOLO detects objects, annotate the frame
            annotated_frame = results[0].plot()
        else:
            # If no detections, use the resized frame as the annotated frame
            annotated_frame = frame_resized.copy()
        
        # Check for car detections and process them
        for result in results:
            boxes = result.boxes.xyxy  # Get the bounding boxes
            classes = result.boxes.cls  # Get the class labels

            for i, box in enumerate(boxes):
                class_id = int(classes[i])
                if class_names[class_id] == "car":  # Only process cars
                    x1, y1, x2, y2 = map(int, box)  # Extract bounding box
                    
                    # Check if the car has crossed the Y-threshold line
                    if has_crossed_threshold(y2) and not car_detected:
                        car_detected = True  # Mark the car as detected
                        print("Car has touched the threshold. Capturing snapshot.")

                        # Crop the car region immediately after crossing the threshold
                        car_region = frame_resized[y1:y2, x1:x2]

                        # Display the cropped car image
                        cv2.imshow("Cropped Car Image", car_region)
                        recognize_plate(car_region)

                        # Wait for a key press to close the window and stop further processing
                        key = cv2.waitKey(0)
                        
                        # Check if the window was successfully opened before destroying it
                        if cv2.getWindowProperty("Cropped Car Image", cv2.WND_PROP_VISIBLE) >= 1:
                            cv2.destroyWindow("Cropped Car Image")

                        return  # Exit after capturing and processing the first car

        # Draw the threshold line on the annotated frame
        cv2.line(annotated_frame, (0, THRESHOLD_LINE_Y), (annotated_frame.shape[1], THRESHOLD_LINE_Y), (0, 255, 0), 2)  # Green line

        # Visualize the result
        cv2.imshow('YOLOv8 Inference', annotated_frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [66]:
if os.path.isfile(input_path) and is_image:
    process_image(frame) 
else:
    capture_and_detect()



0: 448x640 1 car, 31.2ms
Speed: 2.0ms preprocess, 31.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
License Plate Detected: MH 20 EE 7602
Car MH 20 EE 7602 entered at 2024-09-27 11:45:44
Notification sent: Vehicle with Registration No.: MH 20 EE 7602 has entered SP CAR PARKING At 2024-09-27 11:45:44. Thank You
